참고:
http://jaejunyoo.blogspot.com/2017/01/generative-adversarial-nets-1.html
http://jaejunyoo.blogspot.com/2017/01/generative-adversarial-nets-2.html
https://github.com/wiseodd/generative-models

## Vanilla GAN

### 1. 구조
#### a. Generator
 - Image를 만드는 부분
 
#### b. Discriminator
  - 만들어진 image를 평가하는 부분

### - 예
지폐위조범(**Generator**)은 경찰을 최대한 열심히 속이려고하고
다른 한편에서는 경찰(**Discriminator**)이 이렇게 위조된 지폐를 진짜와 감별하려고(Classify) 노력한다.
이런 경쟁 속에서 두 그룹 모두 속이고 구별하는 서로의 능력이 발전하게 되고 결과적으로는 진짜 지폐와 위조 지폐를 구별할 수 없을 정도(구별할 확률 \\(P_d=0.5)\\)에 이른다는 것

### 2. 개요

#### Generative model G는 우리가 갖고 있는 data x의 distribution을 알아내려고 노력한다. 만약 G가 정확히 data distribution을 모사할 수 있다면 거기서 뽑은 sample은 완벽히 data와 구별할 수 없다.
#### 한편 discriminator model D는 현재 자기가 보고 있는 sample이 training data에서 온 것(진짜)인지 혹은 G로부터 만들어진 것인지를 구별하여 각각의 경우에 대한 확률을 estimate한다.

![vanilla_gan](img/vanilla_gan_1.PNG)
D의 입장에서는 data로부터 뽑은 sample x는 D(x)=1이 되고, 임의의 noise distribution으로부터 뽑은 input z를 G에 넣고 만든 sample에 대해서는 D(G(z))=0이 되도록 노력합니다. 즉, D는 실수할 확률을 낮추기(mini) 위해 노력하고 반대로 G는 D가 실수할 확률을 높이기(max) 위해 노력하는데, 둘을 같이 놓고 보면 "**minimax two-player game or minimax problem**"이라 할 수 있다.

### 3. Adversarial Nets

Generator's distribution \\(p_g\\) over data x를 학습하기 위해 generator의 input으로 들어갈 noise variables \\(p_z(z)\\)에 대한 prior를 정의하고, data space로의 mapping을 \\(G(z;\theta_g)\\)라 표현할 수 있습니다. 여기서 G는 미분 가능한 함수로써 \\(\theta_g\\)를 parameter로 갖는 multilayer perceptron입니다.

한편 Discriminator 역시 multilayer perceptron으로 \\(D(x;\theta_d)\\)로 나타내며 output은 확률 single scaler 값이 된다. D(x)는 x가 \\(p_g\\)가 아닌 data distribution으로부터 왔을 확률을 나타낸다.

따라서, 이를 수식으로 정리하면 다음과 같은 value function V(G, D)에 대한 minimax problem을 푸는 것과 같아진다.
$ \min_G \max_D V(D,G) = \mathbb{E}_{x\sim p_{data}~(x)}[log D(x)] + \mathbb{E}_{z\sim p_x(z)}[log(1-D(G(z)))]$

#### 극단적인 예
이상적인 상황에서의 D입장을 살펴보면 D는 아주 잘 구별 하는 녀석이므로 D가 보는 sample x가 실제로 data distribution으로부터 온 녀석이라면 D(x)=1이므로 첫번째 term에서 log 값이 사라지고 G(z)가 만들어낸 녀석이라면 D(G(z))=0이므로 두번째 term역시 0으로 사라진다. 이 때가 D의 입장에서 V의 "최대값"을 얻을 수 있다는 것은 자명하다.

반대로 G의 입장에서 살펴보면 G는 아주 잘 속이는 녀석이므로 D(G(z))=1이 되고 두번째 term은 $ -\infty$가 된다. 이 때 G의 입장에서 V의 "최소값"을 얻을 수 있다.

#### 그림을 통한 예
![example gan](img/vanilla_gan_2.PNG)

검은 점선이 data generating distribution, 파란 점선이 discriminator distribution, 녹색 선이 generative distribution이다. 밑에 x와 z선은 각각 x와 z의 domain을 나타내며 위로 뻗은 화살표가 x=G(z)의 mapping을 보여준다.

처음 시작할 때는 (a)와 같이 $p_g$가 $p_data$와 전혀 다르게 생긴 것을 볼 수 있고 이 상태에서 discriminator가 두 distribution을 구별하기 위해 학습을 하면 (b)와 같이 좀 더 smooth하고 잘 구별하는 distribution이 만들어진다. 이후 G가 현재 discriminator가 구별하기 어려운 방향으로 학습을 하면 (c)와 같이 좀 더 $p_g$가 $p_data$와 가까워지게 되고 이런식으로 쭉 학습을 반복하다보면 결국에는 $p_g=p_data$가 되어 discriminator가 둘을 전혀 구별하지 못하는 즉, $D(x)={1\over2}$인 상태가 된다는 것이다.

#### 실용적인 TIP
위의 value function에서 log(1-D(G(z)))부분을 G에 대해 minimize하는 대신 log(D(G(z)))를 maximize 하도록 G를 학습시킨다.

G가 초기에는 아주 이상한 image들을 생성하기 때문에 D가 너무도 쉽게 이를 real image와 구별하게 되고 따라서 log(1-D(G(z)))값이 0으로 saturate하여 gradient를 계산해보면 아주 작은 값이 나오기 때문에 학습 속도가 매우 느려진다.

하지만 문제를 $G=\arg\max_Glog(D(G(z)))$로 바꾸게 되면 초기에 D가 G로 나온 image를 잘 구별한다고 해도 값이 $-\infty$로 가게 되어 위와 같은 문제가 생기지 않기 때문에 원래 문제와 같은 fixed point를 얻게 되면서도 stronger gradient를 줄 수 있게 된다.

### 4. 이론적 내용

이론적 결과 부분도 크게는 두 가지로 나뉘는데 먼저 앞서 소개한 minimax problem이 $p_g = p_{data}$에서 global optimum을 갖는다는 것을 보이고, 이어서 이 논문에서 소개하는 알고리즘이 global optimum을 찾는다는 것을 보여줍니다.

이 때, 지금까지 잘 사용하던 Multilayer perceptron을 사용하여 내용을 이어가는 것이 아니라 이론적 증명을 편하게 하기 위해 약간의 기믹?을 사용합니다. 저자는 non-parametric setting을 사용했다고 표현하는데, 좀 더 풀어 말하면 MLP를 사용하는 것은 결국 parameter θ를 학습하는 것이므로 직접적으로 probability density function을 학습하는 것과는 차이가 있다는 것이죠.

즉, 앞으로 나올 이론적 증명 등은 model이 infinite capacity를 가지고 있으며, 수렴에 관해 애기할 때도 probability density function 공간에서 얘기한다는 것을 염두에 두면 되겠습니다.

$$ \min_G \max_D V(D,G) = \mathbb{E}_{x\sim p_{data}~(x)}[log D(x)] + \mathbb{E}_{z\sim p_x(z)}[log(1-D(G(z)))] $$

####  Global Optimality of $p_g=p_{data}$

**proposition 1**

For G fixed, the optimal discriminator D is
$$ D_{G}^{*}(x) = {p_{data}(x)\over P_{data}(x)+p_g(x)}$$

Proof. The training criterion for the discriminator D, given any generator G, is to maximize the quantity V(G,D)
$$ \begin{align}V(G,D) &= \int_x p_{data}~(x)log(D(x))dx + \int_z p_z(z)log(1-D(G(z)))dz \\ &= \int_x p_{data}~(x)log(D(x)) + p_g(x)log(1-D(x))dx \end{align} $$
For any $(a,b) \in \mathbb{R}^2 \setminus \{0,0\}$, the function $y \rightarrow {\rm a} log(y) + {\rm b} log(1-y)$ achieves its maximum in [0,1] at $\frac{a}{a+b}$. The discriminator does not need to be defined outside of $Supp(p_{data}) \cup Supp(p_g)$, concluding the proof.

결국  $\min_G \max_D V(G,D)$에서 안 쪽의 max 문제부터 풀어주면 문제가 다음과 같이 reformulate 됩니다:
$$ \begin{align} C(G) & = \max_D V(G,D) \\ 
&= \mathbb{E}_{x \sim p_{data}} \left[ log D^*_G(x) \right] + \mathbb{E}_{z\sim p_z}\left[ log(1-D^*_G(G(z))) \right] \\ 
&= \mathbb{E}_{x \sim p_{data}} \left[ log D^*_G(x) \right] + \mathbb{E}_{x\sim p_g}\left[ log(1-D^*_G(x)) \right] \\ 
&= \mathbb{E}_{x \sim p_{data}} \left[ log \frac{p_{data}~(x)}{p_{data}~(x)+p_{g}(x)} \right] + \mathbb{E}_{x\sim p_g} \left[ log \frac{p_{g}(x)}{p_{data}~(x)+p_{g}(x)} \right] \end{align} $$

___

#### Theorem 1.
The global minimum of the virtual training criterion C(G) is achieved if and only if $p_g=p_{data}$. At that point, C(G) achieves the value −log(4).

##### Proof.
For $p_g = p_{data}$, $D_G^*(x)={1\over2}$ 임은 자명하고 다음 수식도 자연히 따라온다.
$$C(G) = \mathbb{E}_{x \sim p_{data}} \left[ -log(2)\right] + \mathbb{E}_{x \sim p_{g}} \left[ -log(2)\right]=-log(4).$$

이 값이 best possible value of C(G)란 것을 알기 위해서는 다음과 같이 C(G)를 표현하는 것이 증명의 키 입니다:

\begin{align}C(G) &= C(G) + log(4) -log(4) \\ &= \mathbb{E}_{x \sim p_{data}} \left[ log \frac{p_{data}~(x)}{p_{data}~(x)+p_{g}(x)} \right] + \mathbb{E}_{z\sim p_x(z)} \left[ log \frac{p_{g}(x)}{p_{data}~(x)+p_{g}(x)} \right] + log(2) + log(2) - log(4) \\ &= -log(4) + KL \left( p_{data} || \frac{p_{data}~+ p_g}{2}\right) + KL \left( p_g|| \frac{p_{data}~+ p_g}{2}\right) \\ &= -log(4) + 2\cdot JSD(p_{data}||p_g) \end{align}

여기서 KL은 Kullback-Leibler divergence이고 JSD는 Jensen-Shannon divergence입니다. JSD는 항상 양수이고 두 distribution이 일치할 때만 0이므로 $C^∗=−log(4)$가 C(G)의 global minimum이며 그 유일한 해가 $p_g=p_{data}$임을 알 수 있습니다.

---

**참고 **:

**- Kullback-Leibler divergence**
$$ D_{KL}(P||Q) = \sum_i P(i) log\frac{P(i)}{Q(i)}$$
- P라는 distribution이 있을 때 (보통은 estimate한)Q가 P랑 얼마나 다른지를 측정하는 값

**- Jensen-Shannon divergence**
$$ {\rm JSD}(P \parallel Q)= \frac{1}{2}D(P \parallel M)+\frac{1}{2}D(Q \parallel M)$$

---

#### Convergence of Algorithm 1

minimax problem을 잘 풀기만 하면 (즉, global optimal을 찾으면), generator가 만드는 probability distribution($p_g$)이 data distribution($p_{data}$)과 정확히 일치하도록 할 수 있다

이 논문에서는 이미  신경망 모델(MLP)을 사용하여 G와 D를 정의하고 각각을 fix한 상태에서 번갈아가며 문제를 풀어주는 전략을 제시했기 때문에 이제 남은 것은 제시한 알고리즘이 문제를 잘 풀어주는가? 혹은 Global Optimum인 $p_g=p_{data}$로 수렴하는가?를 확인하는 것입니다. 

**Proposition**

If G and D have enough capacity, and at each step of Algorithm 1, the discriminator is allowed to reach its optimum given G, and $p_g$ is updated so as to improve the criterion
$$ \mathbb{E}_{x \sim p_{data}} \left[ log D^*_G(x) \right] + \mathbb{E}_{x\sim p_g}\left[ log(1-D^*_G(x)) \right] $$
then $p_g$ converges to $p_{data}$.

<span style="color:red">** Proof.**  Consider $V(G,D)=U(p_g,D)$ as a function of $p_g$ as done in the above criterion. Note that $U(p_g,D)$ is convex in $p_g$. The subderivatives of a supremum of convex functions include the derivative of the function at the point where the maximum is attained. This is equivalent to computing a gradient descent update for pg at the optimal D given the corresponding G. $sup_DU(p_g,D)$ is convex in $p_g$ with a unique global optima as proven in Thm 1, therefore with sufficiently small updates of $p_g$, $p_g$ converges to $p_x$, concluding the proof.</span>

"In practice, adversarial nets represent a limited family of $p_g$ distributions via the function $G(z;θ_g)$, and we optimize $θ_g$ rather than $p_g$ itself."

### Tensorflow Code

In [6]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

ERROR:root:Line magic function `%inline` not found.


In [7]:
%matplotlib inline

In [2]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

In [3]:
X = tf.placeholder(tf.float32, shape=[None, 784])

D_W1 = tf.Variable(xavier_init([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


Z = tf.placeholder(tf.float32, shape=[None, 100])

G_W1 = tf.Variable(xavier_init([100, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(xavier_init([128, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [4]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob


def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit

In [5]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [13]:
G_sample = generator(Z)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
G_loss = -tf.reduce_mean(tf.log(D_fake))

# Alternative losses:
# -------------------
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
#D_loss = D_loss_real + D_loss_fake
#G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

mb_size = 128
Z_dim = 100

mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

Extracting ../../MNIST_data\train-images-idx3-ubyte.gz
Extracting ../../MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data\t10k-labels-idx1-ubyte.gz


In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')

i = 0

In [16]:
for it in range(10000):
    if it % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})

        fig = plot(samples)
        plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(mb_size, Z_dim)})

    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

c:\program files\anaconda3\lib\site-packages\matplotlib\colors.py:943: UserWarning: Warning: converting a masked element to nan.
  vmin = float(vmin)
c:\program files\anaconda3\lib\site-packages\matplotlib\colors.py:944: UserWarning: Warning: converting a masked element to nan.
  vmax = float(vmax)


Iter: 0
D loss: 1.386
G_loss: 0.6931

Iter: 1000
D loss: 1.386
G_loss: 0.6931

Iter: 2000
D loss: 1.386
G_loss: 0.6931

Iter: 3000
D loss: 1.386
G_loss: 0.6931

Iter: 4000
D loss: 1.386
G_loss: 0.6931

Iter: 5000
D loss: 1.386
G_loss: 0.6931

Iter: 6000
D loss: 1.386
G_loss: 0.6931

Iter: 7000
D loss: 1.386
G_loss: 0.6931



KeyboardInterrupt: 

![img000](out/000.png)
![img001](out/001.png)
![img002](out/002.png)
![img003](out/003.png)
![img004](out/004.png)
![img005](out/005.png)
![img006](out/006.png)
![img007](out/007.png)
![img008](out/008.png)
![img009](out/009.png)